In [1]:
import os
import torch
from skimage.io import imread
from skimage import img_as_ubyte
from sigVrfy.utils import preprocess_signature
from sigVrfy.models.signet import SigNet

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: {}".format(device))

Using device: cpu


In [3]:
SIG_MODEL_PATH = '../data/pretrain_models/signet.pth'

In [4]:
state_dict, _, _ = torch.load(SIG_MODEL_PATH)
base_model = SigNet().to(device).eval()
base_model.load_state_dict(state_dict)

C:\Users\PC\AppData\Local\Temp\ipykernel_9356\2478507757.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict, _, _ = torch.load(SIG_MODEL_PATH)


<All keys matched successfully>

In [5]:
def load_signature(path):
    return img_as_ubyte(imread(path, as_gray=True))

In [6]:
def compare_signatures(images_path, canvas_size=(952, 1360)):
    user_signs = [load_signature(path) for path in images_path]
    processed_user_sigs = torch.tensor(
        [preprocess_signature(sig, canvas_size) for sig in user_signs]
    )
    processed_user_sigs_scaled = (
        processed_user_sigs.view(-1, 1, 150, 220).float().div(255)
    )
    with torch.no_grad():
        user_features = base_model(processed_user_sigs_scaled.to(device))

    distance = torch.norm(user_features[0] - user_features[1])
    print(f"Euclidean distance between signatures from the same user: {distance}")

    if distance < 15.5:
        return "Original"
    else:
        return "Forged"

In [7]:
compare_signatures(['../data/test_images/a1.png', '../data/test_images/a2.png'])

C:\Users\PC\AppData\Local\Temp\ipykernel_9356\1645814978.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  processed_user_sigs = torch.tensor(


Euclidean distance between signatures from the same user: 15.390494346618652


'Original'